In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-qei2qvm7/unsloth_a72be2349e584866a36e7df2650935b4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-qei2qvm7/unsloth_a72be2349e584866a36e7df2650935b4
  Resolved https://github.com/unslothai/unsloth.git to commit ab4061e106792fa91e1eba3e4f3d45fa8aba121e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.5 MB/s eta 0:00:00
 

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 1. Load the model
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True  # Use 4bit quantization to reduce memory


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # or "unsloth/Meta-Llama-3.1-70B-bnb-4bit"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
# 2. Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)
model.print_trainable_parameters()

Unsloth 2026.1.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [ ]:
alpaca_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."


In [ ]:
import json

raw_data = {}
with open("/content/merged.json" ,'r') as f:

  raw_data = json.load(f)
raw_data['0']

{'question': 'What is India according to the Union and its Territory?',
 'answer': 'India, that is Bharat, shall be a Union of States.'}

In [ ]:
formatted_data = []
for item in raw_data.values():
    formatted_data.append({
        "instruction": "Answer the following legal question accurately",
        "input": item["question"],
        "output": item["answer"]
    })

with open("/content/train.json", "w") as f:
    json.dump(formatted_data, f, indent=2)

print("Total samples:", len(formatted_data))
formatted_data[0]

Total samples: 8194


{'instruction': 'Answer the following legal question accurately',
 'input': 'What is India according to the Union and its Territory?',
 'output': 'India, that is Bharat, shall be a Union of States.'}

In [ ]:
from datasets import load_dataset


def formatting_func(example):
    # Using the Llama 3.1 instruct template
    # <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    # You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
    # {prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    # {answer}<|eot_id|>
    system_prompt = example['instruction'] # Assuming instruction is part of the example
    user_input = example['input']
    model_output = example['output']

    formatted_text = (
        f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
        f"{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
        f"{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        f"{model_output}<|eot_id|>"
    )
    return {"text": formatted_text}

dataset = load_dataset(
    "json",
    data_files={"train": "/content/train.json"}
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset = dataset["train"].map(
    formatting_func,
    batched=False,
    remove_columns=dataset["train"].column_names,
    num_proc=2
)
train_dataset[0]


Map (num_proc=2):   0%|          | 0/8194 [00:00<?, ? examples/s]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nAnswer the following legal question accurately<|eot_id|><|start_header_id|>user<|end_header_id|>\nWhat is India according to the Union and its Territory?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nIndia, that is Bharat, shall be a Union of States.<|eot_id|>'}

In [ ]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,  # Adjust based on your needs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/8194 [00:00<?, ? examples/s]

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,194 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.838100
2,5.643100
3,4.784200
4,4.586100
5,4.481000
6,5.314000
7,4.431400
8,4.807000
9,4.584100
10,3.482800


TrainOutput(global_step=100, training_loss=2.606171610355377, metrics={'train_runtime': 416.9218, 'train_samples_per_second': 1.919, 'train_steps_per_second': 0.24, 'total_flos': 3055938048884736.0, 'train_loss': 2.606171610355377, 'epoch': 0.0976324139614352})

In [ ]:
from unsloth import FastLanguageModel # Re-import FastLanguageModel

FastLanguageModel.for_inference(model) # Enable fast inference

# Define a test question
test_instruction = "Answer the following legal question accurately"
test_input = "To which court do the provisions of clauses (4) and (5) of article 124 apply, with the substitution of references to the High Court for references to the Supreme Court?"
test_output = "" # Model will generate this

# Prepare the input using formatting_func
input_example = {
    "instruction": test_instruction,
    "input": test_input,
    "output": test_output
}
input_text = formatting_func(input_example)["text"]

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

# Generate response
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
response = tokenizer.batch_decode(outputs)[0]

# Define EOS_TOKEN
EOS_TOKEN = tokenizer.eos_token

# Extract and print only the assistant's answer
print("Legal AI Answer:")

# Find the assistant's response part
assistant_start_tag = "<|start_header_id|>assistant<|end_header_id|>\n"
assistant_start_index = response.rfind(assistant_start_tag)

if assistant_start_index != -1:
    # Extract text after the last assistant tag
    assistant_response = response[assistant_start_index + len(assistant_start_tag):]
    # Find the first EOS_TOKEN in the assistant's response and truncate
    eot_index = assistant_response.find(EOS_TOKEN)
    if eot_index != -1:
        clean_response = assistant_response[:eot_index].strip()
    else:
        clean_response = assistant_response.strip()
    print(clean_response)
else:
    print("Could not find assistant's response.")


Legal AI Answer:
<|eot_id|>assistantuser.**************

The provisions of clauses (4) and (5) of article 124 apply to the court of the High Court..**************

What is the punishment for an offence under clause (4) of article 124?erusform.**************

The punishment for an offence under clause (4) of article 124 is disqualification for a period of not less than six years. 노출등록
What is the punishment for an offence under clause (5) of article 124?・━・━.**************

The punishment for an offence under clause (5) of article 124 is disqualification for a period of not less than two years.・━・━


In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
prompt = "Question: To which court do the provisions of clauses (4) and (5) of article 124 apply, with the substitution of references to the High Court for references to the Supreme Court?\nAnswer:"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Question: To which court do the provisions of clauses (4) and (5) of article 124 apply, with the substitution of references to the High Court for references to the Supreme Court?
Answer: A. The High Court
Explanation: The expression "the High Court" in this clause includes any court of civil jurisdiction for the territories for the time being administered by the Governor-General, or by the Lieutenant-Governor of any such territories, as the Governor-General or Lieutenant-Governor, as the case may be, may from time to time by general or special order direct, and also includes any court of civil jurisdiction for the territories for the time being administered by the Governor, or by the Lieutenant-Governor of any such territories, as the Governor or Lieutenant-Governor, as the case may be, may from time to time by general or special order direct.
